In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

# jax
import jax.numpy as jnp
import time

import sys
import os
import importlib
import pickle
import itertools

def append_path(path):
    if path not in sys.path:
        sys.path.append(path)

def reload_module(module_name, class_name):
    module = importlib.import_module(module_name)
    importlib.reload(module)
    return getattr(module, class_name)
        
append_path(os.path.abspath(os.path.join('..', '00_utils')))
append_path(os.path.abspath(os.path.join('..', '00_utils_training')))
append_path(os.path.abspath(os.path.join('..', '00_models')))

import logging
logging.basicConfig(level=logging.ERROR, filename='error_log.txt')

import optimize_pyomo_rl

* [X] Regularization Strength with Fixed Tolerance
* [X] 'tol' with no regularization
* [X] 'tol - 'reg' cross-validation

In [96]:
param_combinations_input_features = [{'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':1,'ls': [8, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':1, 'ls': [6, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':1, 'ls': [5, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':True, 'm':0,'ls': [7, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [6, 32, 1]}, 
                        {'prev_hour':True, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':True, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':True, 'prev_year':False, 'm':0, 'ls': [5, 32, 1]}, 
                        {'prev_hour':False, 'prev_week':False, 'prev_year':False, 'm':0, 'ls': [4, 32, 1]}, 
                        ]

In [97]:
ExperimentRunner = reload_module('optimize_pyomo_rl', 'ExperimentRunner')

tol = 1e-8
start_date = '2015-01-10'
extra_input = {}
extra_input['params_data'] = {'file_path': '../00_data/df_train.csv', 'start_date': start_date, 
                'n_points': 300, 'split': 200, 'n_days': 1, 'm': 1, 
                'prev_hour': False, 'prev_week': True, 'prev_year': True,
                'spacing': 'gauss_radau',
                #'spacing': 'chebyshev',
                'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'},}

extra_input['params_sequence'] = {'sequence_len': 10, 'frequency': 35}
extra_input['params_model'] = {'layer_sizes': [7, 32, 1], 'penalty': 1e-7, 'w_init_method': 'xavier'}
extra_input['params_solver'] = { 
                        "tol":tol, 
                        "dual_inf_tol": 0.1, 
                        "compl_inf_tol": tol,
                        "constr_viol_tol": 1e-4, 
                        'warm_start_init_point': 'yes',
                        "halt_on_ampl_error" : 'yes',
                        "print_level": 5, "max_iter": 3000,
                        "bound_relax_factor": 1e-8}

extra_input['plot_collocation'] = True
extra_input['plot_odeint'] = True
extra_input['convergence_steps'] = 5

runner = ExperimentRunner(start_date, 'collocation_method', extra_input)
runner.run()

Generating default parameters for ode solver
Running iteration 1 with parameters: chebyshev
{'layer_sizes': [7, 32, 1], 'penalty': 1e-07, 'w_init_method': 'xavier'}
Spacing type: chebyshev
w init method: xavier
current_16_08
y
Ipopt 3.14.16: tol=1e-08
dual_inf_tol=0.1
compl_inf_tol=1e-08
constr_viol_tol=0.0001
warm_start_init_point=yes
halt_on_ampl_error=yes
print_level=5
max_iter=3000
bound_relax_factor=1e-08


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:    97280
Number of nonzeros in inequality constraint Jacobian.:        

In [99]:
runner.results_avg

{'chebyshev': {'times_elapsed': 7.628415060043335,
  'mse_odeint': 313.49131274157935,
  'mse_coll_ode': 0.2017300999411741,
  'mse_odeint_test': 159.94367152390822,
  'mse_coll_ode_test': 0.7200797479223991},
 'gauss_legendre': {'times_elapsed': 8.542209053039551,
  'mse_odeint': 1236.7934794065702,
  'mse_coll_ode': 0.1721682388037794,
  'mse_odeint_test': 365.2177201865353,
  'mse_coll_ode_test': 0.6184263793533125},
 'gauss_radau': {'times_elapsed': 8.622624397277832,
  'mse_odeint': 202.82180576681807,
  'mse_coll_ode': 0.03568684073165636,
  'mse_odeint_test': 101.47637962169242,
  'mse_coll_ode_test': 1.232583487223685},
 'gauss_lobatto': {'times_elapsed': 9.25119616985321,
  'mse_odeint': 585.5312510954569,
  'mse_coll_ode': 0.1249785560463568,
  'mse_odeint_test': 246.10636540894475,
  'mse_coll_ode_test': 0.6138424156952406}}

In [90]:
runner.results_full

{(1,
  '2015-01-10'): {'result': {'solver_time': 6.334311008453369,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 6.334311008453369, 'mse_odeint': 1299.0604050855536, 'mse_coll_ode': 0.010894143036727623, 'mse_odeint_test': 310.8754126479351, 'mse_coll_ode_test': 0.3540610985737233},
 (1,
  '2015-02-14'): {'result': {'solver_time': 9.612164974212646,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 9.612164974212646, 'mse_odeint': 0.005715961159941665, 'mse_coll_ode': 0.0064556898206132906, 'mse_odeint_test': 0.1598123795540598, 'mse_coll_ode_test': 0.162570335264609},
 (1,
  '2015-03-21'): {'result': {'solver_time': 7.228785276412964,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 7.22878

In [85]:
runner.results_avg

{1: {'times_elapsed': 7.046317052841187,
  'mse_odeint': 372.62027318280826,
  'mse_coll_ode': 0.05570399430442987,
  'mse_odeint_test': 128.96098064574062,
  'mse_coll_ode_test': 0.8033615429788682}}

----

In [27]:
runner.save_trained_weights('rl_32_weights')

Results saved to ../00_trained_wb/rl_32_weights_2024-09-02_12-09-44.pkl


In [91]:
runner.save_results('short_training_10')

Results saved to results/short_training_10_2024-09-08_13-37-35_full.pkl
Results saved to results/short_training_10_2024-09-08_13-37-35_avg.pkl
